### 基本内容
1. 数据收集与分类
2. 数据解析与内容提取
3. 数据清洗与标准化
4. 数据存储与索引 
5. 后续处理与验证

### 数据收集与分类
- 目标：整理原始文件，按类型分类
  - 创建统一文件夹（如 documents/, audio/, video/）
  - 使用脚本自动分类（Python + os 库按扩展名分类）
  - 记录元数据（文件名、格式、大小、创建时间）

### 数据解析与内容提取
通用原则：将非文本数据（音频/视频）转为文本，提取文档中的结构化/非结构化内容

#### 1. 文档类处理

##### Word
- 使用 python-docx 提取文本和表格

In [2]:
from docx import Document
import re
doc = Document("../files/doc.docx")
text = "\n".join([para.text for para in doc.paragraphs])
re.sub(r'\s+', ' ', text)

'智会 AiMeeting 产品简介 系统介绍 智会AiMeeting是专为金融行业定制的会议协作管理平台，专注于提升办公效率和服务竞争力。 平台囊括策略会流程中的会议管理、邀请函管理、报名管理、日程管理、现场签到、会后数据统计等功能，信息流通无边界，报名、日程无时差；针对策略会排会工作量大的特点，平台提供智能的自动排会算法和丰富的手动排会工具，最大限度满足报名公司及客户需求。 中台、研究员、销售协同合作，支持客户自主报名，共同提升办会效率。 合作券商 （按合作时间先后排名） 天风证券 华创证券 招商证券 光大证券 方正证券 中信建投 开源证券 海通证券 德邦证券 民生证券 财通证券 东方证券 国泰君安 上海证券 华福证券 华源证券 国金证券 优势介绍 智会AiMeeting的研发团队与券商深度合作，探索需求、打磨产品、匹配不同规模的策略会场景，是专为金融行业定制的会议协作管理平台。 针对策略会报名信息变动频繁的特点，客户报名方面，与公司的参会时间解耦，客户报名时仅需预约公司，不受公司时间变动的影响；信息流通方面，研究员与销售可以在平台实时查看最新信息；排会方面，平台提供增量排会的模式，可以在尽量保持原有日程不变的前提下快速匹配最新需求。 对于排会结果，公司或客户需求若未能满足，可以在平台查看未能匹配的原因，以作相应调整。 针对券商对研究员/销售在系统操作权限的控制需求，平台将系统的操作点细分为权限项，管理员可以根据不同阶段的需求对研究员/销售角色设置相应的权限；数据层面，研究员/销售默认只能更新个人负责的信息，此外增加组长的设置，可以管理同组员的信息。 中台、研究员、销售通过权限控制在同一平台负责各自内容，协同合作，共同提升办会效率。'

##### Excel
- 使用 pandas 读取数据，保存为CSV或JSON
- 依赖 openpyxl 库

In [15]:
import pandas as pd
df = pd.read_excel("../files/excel.xlsx")
print(df.head())

  上市公司名称       股票代码  参会人数    研究员 参会类型  行业 是否线上参会  状态  备注
0   平安银行  000001.sz     1  农业-组员  小范围  农业      是  通过 NaN
1   国华网安  000004.sz     1   研究员C  小范围  机械      否  通过 NaN


##### PDF
- 使用 PyMuPDFLoader 来读取 PDF 文件
    - PyMuPDFLoader 是 PDF 解析器中速度最快的一种
    - 结果会包含 PDF 及其页面的详细元数据，并且每页返回一个文档。
- 示例代码（文本提取）：

In [16]:
from langchain.document_loaders import PyMuPDFLoader

# 创建一个 PyMuPDFLoader Class 实例，输入为待加载的 pdf 文档路径
ploader = PyMuPDFLoader("../files/pumpkin_book.pdf")

# 调用 PyMuPDFLoader Class 的函数 load 对 pdf 文件进行加载
pages = ploader.load()

print(f"载入后的变量类型为：{type(pages)}",  f"该 PDF 一共包含 {len(pages)} 页", sep="\n")

载入后的变量类型为：<class 'list'>
该 PDF 一共包含 196 页


In [17]:
page = pages[1]
print(f"该元素的类型：{type(page)}.", 
    f"该文档的描述性数据：{page.metadata}", 
    f"该文档的字数: {len(page.page_content)}", 
    f"该文档的内容:\n{page.page_content[0:300]}", 
    sep="\n------\n")

该元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'producer': 'xdvipdfmx (20200315)', 'creator': 'LaTeX with hyperref', 'creationdate': '2023-11-17T15:20:45+00:00', 'source': '../files/pumpkin_book.pdf', 'file_path': '../files/pumpkin_book.pdf', 'total_pages': 196, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20231117152045-00'00'", 'page': 1}
------
该文档的字数: 1313
------
该文档的内容:
前言
“周志华老师的《机器学习》（西瓜书）是机器学习领域的经典入门教材之一，周老师为了使尽可能多的读
者通过西瓜书对机器学习有所了解, 所以在书中对部分公式的推导细节没有详述，但是这对那些想深究公式推
导细节的读者来说可能“不太友好”，本书旨在对西瓜书里比较难理解的公式加以解析，以及对部分公式补充
具体的推导细节。”
读到这里，大家可能会疑问为啥前面这段话加了引号，因为这只是我们最初的遐想，后来我们了解到，周
老师之所以省去这些推导细节的真实原因是，他本尊认为“理工科数学基础扎实点的大二下学生应该对西瓜书
中的推导细节无困难吧，要点在书里都有了，略去的细节应能脑补或做练习”。所以...... 


##### Markdown
- 使用 LangChain 提供的 UnstructuredMarkdownLoader
- 将 Markdown 文件中的内容转换为 LangChain 的文档对象，方便后续的文本处理和分析
- 能够识别并解析 Markdown 中的各种元素，如标题、段落、列表、链接等

#### 2. 音频/视频处理

##### 视频预处理
- 使用 FFmpeg 提取音频：
```
ffmpeg -i video.mp4 -vn -acodec libmp3lame audio.mp3
```

##### 语音转文本
- 将音视频文件直接转为 text 文件再通过代码读取
    - ```whisper ./video.mp4 --model large --model_dir whisper-large --language zh --output_dir ./output```
- 代码读取音频文件：

In [20]:
import whisper
model = whisper.load_model("base")
result = model.transcribe("../files/audio.mp3", language='zh')
print(result["text"])

/Users/livia/miniforge3/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


同時們大家好,這節刻我們要講的是時間複雜度計算首先,我們來看一下時間複雜度是什麼簡單來說,時間複雜度就是一個韓數與其他韓數所不同,它是用大寫的歐來表示的比如說歐1、歐N、歐LogN等等


### 数据清洗与标准化

#### 数据处理三件套

- Pandas
    - 数据清洗：`dropna()`删除缺失值、`replace()`替换异常值、`duplicated()`标记重复行
    - 数据分割：`pd.cut()`等宽分箱、`pd.qcut()`等频分箱、`groupby`分组统计
    - 数据合并：`concat()`轴向堆叠、`merge()`主键合并
- Numpy
    - 高效处理数组：向量化运算加速数据转换（如`np.where`条件筛选）
- Scikit-learn
    - 标准化工具：`StandardScaler（Z-score）`、`MinMaxScaler`（归一化）
    - 数据分割：`train_test_split`划分训练集/测试集

#### 路径规划
- 基础：掌握`Pandas`、`Numpy`核心操作、缺失值/重复值处理等
- 核心：熟练数据分箱、标准化、合并技巧
- 实战：完成数据预处理项目
- 进阶：构建数据清洗、标准化自动化流水线